## Hello Kagglers, in this Notebook we will try to use TabNet Regressor to solve TPS Playground problem! I added some credits in the final of this notebook :D

## TabNet: Attentive Interpretable Tabular Learning

"We propose a novel high-performance and interpretable canonical deep tabular data learning architecture, TabNet. TabNet uses sequential attention to choose which features to reason from at each decision step, enabling interpretability and more efficient learning as the learning capacity is used for the most salient features. We demonstrate that TabNet outperforms other neural network and decision tree variants on a wide range of non-performance-saturated tabular datasets and yields interpretable feature attributions plus insights into the global model behavior. Finally, for the first time to our knowledge, we demonstrate self-supervised learning for tabular data, significantly improving performance with unsupervised representation learning when unlabeled data is abundant." 

[TabNet Paper](https://arxiv.org/abs/1908.07442)


## Install PyTorch TabNet

In [ ]:
!pip install pytorch-tabnet

## Parameters

In [ ]:
FLAG_LOCAL = False # Flag to run in kaggle notebook or in jupyter server :)

KAGGLE_PATH = "/kaggle/input/tabular-playground-series-feb-2021/"
LOCAL_PATH = "/home/rapela/Downloads/kaggle/tps_feb/input/"
PATH = (LOCAL_PATH if FLAG_LOCAL else KAGGLE_PATH)

TRAIN_PATH = PATH + "train.csv"
TEST_PATH = PATH + "test.csv"
SUBMISSION = PATH + "sample_submission.csv"
SUBMISSION_OUTPUT = "submission.csv"

print(TRAIN_PATH)
print(TEST_PATH)
print(SUBMISSION)

NUM_FOLDS = 2
SEED = 42

## TabNet Parameters

In [ ]:
## TabNet Parameters
MAX_EPOCH = 1000
N_D = 8 
N_A = 8 
N_STEPS = 2
GAMMA = 1.1
LAMBDA_SPARSE = 0
OPT_LR = 1e-3
OPT_WEIGHT_DECAY = 1e-5
OPT_MOMENTUM = 0.9
MASK_TYPE = "entmax"
SCHEDULER_MIN_LR = 1e-6
SCHEDULER_FACTOR = 0.9
DEVICE_NAME = "cuda"

BATCH_SIZE = 512

## Imports Libs

In [ ]:
import torch
from torch import nn
from pytorch_tabnet.tab_model import TabNetRegressor

from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd 

import os
import random
import sys
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(SEED)

## Import Data

In [ ]:
train = pd.read_csv(TRAIN_PATH, index_col='id')
test = pd.read_csv(TEST_PATH, index_col='id')
submission = pd.read_csv(SUBMISSION, index_col='id')
target = train.pop("target")
target = target.values

## Label Encoding

In [ ]:
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        
        train[c] = lbl.transform(train[c].values)
        test[c] = lbl.transform(test[c].values)

In [ ]:
columns = test.columns

## Create TabNet Params Dictionary

In [ ]:
tabnet_params = dict(n_d=N_D, n_a=N_A, n_steps=N_STEPS, gamma=GAMMA,
                     lambda_sparse=LAMBDA_SPARSE, optimizer_fn=torch.optim.SGD,
                     optimizer_params=dict(lr=OPT_LR, weight_decay=OPT_WEIGHT_DECAY, momentum=OPT_MOMENTUM),
                     mask_type=MASK_TYPE,
                     scheduler_params=dict(mode="min",
                                           patience=20,
                                           min_lr=SCHEDULER_MIN_LR,
                                           factor=SCHEDULER_FACTOR,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     device_name=DEVICE_NAME,
                     seed=SEED
                     )

## Run Kfold with TabNet Regressor

In [ ]:
train_oof = np.zeros((len(train)))
test_preds = 0

kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):

    print(f'Fold {f}')
    train_df, val_df = train.iloc[train_ind][columns], train.iloc[val_ind][columns]

    train_target, val_target = target[train_ind], target[val_ind]

    print(train_df.shape, train_target.shape)
    print(val_df.shape, val_target.shape)

    train_target=train_target.reshape(-1,1)
    val_target=val_target.reshape(-1,1)

    train_df      = train_df.to_numpy()
    train_target      = train_target.reshape(-1, 1)

    val_df = val_df.to_numpy()
    val_target = val_target.reshape(-1, 1)

    model = TabNetRegressor(**tabnet_params)

    model.fit(X_train=train_df,
              y_train=train_target,
              eval_set=[(val_df, val_target)],
              eval_name = ["val"],
              eval_metric = ['mse'],#["logits_ll"],
              max_epochs=MAX_EPOCH, #20
              patience=20, batch_size=BATCH_SIZE,
              num_workers=1, drop_last=False)#,

    temp_oof = model.predict(val_df)
    train_oof[val_ind] = temp_oof.reshape(-1)
    temp_test = model.predict(test.to_numpy())

    test_preds += temp_test/NUM_FOLDS     

    print(mean_squared_error(temp_oof, val_target, squared=False))




## Submit your output csv

In [ ]:
submission['target'] = test_preds
submission.to_csv(SUBMISSION_OUTPUT)

## Credits

* [TabNet Paper](https://arxiv.org/pdf/1908.07442.pdf)
* [TabNet PyTorch GitHub](https://github.com/dreamquark-ai/tabnet)
* [Kaggle Notebook TabNet Regressor](https://www.kaggle.com/optimo/tabnetregressor-2-0-train-infer?scriptVersionId=44853427)
* [Tunguz CV Notebook](https://www.kaggle.com/tunguz/tps-02-21-feature-importance-with-xgboost-and-shap)

## Some ideas:

* Fork the notebook and try to change some parameters and play with the model...
* Try to add some preprocessing...
* Try other encoding than label encoding...
* Ensemble your model with others to make it more diversity...

## If it was useful for you please comment! Your feedback is really important! 